In [15]:
%load_ext autoreload

In [20]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    b_w = [(0,6), (6,12)]
    a_w = [(0,4), (4,8), (8,12)]
    before_windows = b_w
    after_windows = a_w
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [4]:
vital_signs = ['Heart Rate', 'Respiratory rate','Oxygen saturation', 'Systolic blood pressure', 'Diastolic blood pressure', 'Temperature']
labs_bmp = ['Glucose','Potassium','Sodium','Chloride', 'Creatinine', 'Blood urea nitrogen', 'Bicarbonate', 'Calcium',
'Albumin', 'Lactate dehydrogenase','Magnesium','Lactic acid']
labs_cbc = ['Hematocrit','Hemoglobin', 'Platelets', 'White blood cell count', 'Red blood cell count',
'Mean corpuscular volume', 'Lymphocytes', 'Neutrophils']
labs_cauglation = ['Prothrombin time INR']

In [ ]:
# h_labs = pd.read_csv("temp_hirid_labs.csv").drop(columns=["Unnamed: 0"])
# h_med1 = pd.read_csv("temp_hirid_med1.csv")
# h_med2 = pd.read_csv("temp_hirid_med2.csv")

In [5]:
# HIRID
hirid_mapping = constants.HIRID_MAPPING
lab_parts = (0,10)
hirid_parser = hirid.HiRiDParser(data=raw_path, res=res_path, gender=gender, age_b=age_b, age_a=age_a, load="MANUAL_MAPPING_HIRID")
h_med1, h_med2, h_labs = hirid_parser.parse(lab_parts=lab_parts)

In [9]:
lab_with_index = h_labs[["HADM_ID", "AGE", "GENDER", "LabTimeFromAdmit", "VALUENUM", "ITEMID"]].pivot_table("VALUENUM", ["HADM_ID", "AGE", "GENDER",  "LabTimeFromAdmit"], "ITEMID")
lab_with_index

ITEMID                                     Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma  \
HADM_ID AGE GENDER LabTimeFromAdmit                                                                                  
4       75  M      0 days 00:00:00                                                       NaN                         
                   0 days 00:05:40.930000                                                NaN                         
                   0 days 00:05:40.940000                                                NaN                         
                   0 days 00:06:47.980000                                                NaN                         
                   0 days 00:06:48                                                       NaN                         
...                                                                                      ...                         
33904   70  F      5 days 12:50:27.840000                                                NaN                         
                   5 days 12:52:06.110000                                                NaN                         
                   5 days 12:52:06.740000                                                NaN                         
                   5 days 12:53:41.210000                                                NaN                         
                   5 days 12:54:14.280000                                                NaN                         

ITEMID                                     Amylase [Enzymatic activity/volume] in Body fluid  \
HADM_ID AGE GENDER LabTimeFromAdmit                                                            
4       75  M      0 days 00:00:00                                                       NaN   
                   0 days 00:05:40.930000                                                NaN   
                   0 days 00:05:40.940000                                                NaN   
                   0 days 00:06:47.980000                                                NaN   
                   0 days 00:06:48                                                       NaN   
...                                                                                      ...   
33904   70  F      5 days 12:50:27.840000                                                NaN   
                   5 days 12:52:06.110000                                                NaN   
                   5 days 12:52:06.740000                                                NaN   
                   5 days 12:53:41.210000                                                NaN   
                   5 days 12:54:14.280000                                                NaN   

ITEMID                                     Amylase [Enzymatic activity/volume] in Serum or Plasma  \
HADM_ID AGE GENDER LabTimeFromAdmit                                                                 
4       75  M      0 days 00:00:00                                                       NaN        
                   0 days 00:05:40.930000                                                NaN        
                   0 days 00:05:40.940000                                                NaN        
                   0 days 00:06:47.980000                                                NaN        
                   0 days 00:06:48                                                       NaN        
...                                                                                      ...        
33904   70  F      5 days 12:50:27.840000                                                NaN        
                   5 days 12:52:06.110000                                                NaN        
                   5 days 12:52:06.740000                                                NaN        
                   5 days 12:53:41.210000                                                NaN        
    

In [11]:
lab_with_index.to_csv(os.path.join(res_path, f"hirid_extract_{lab_parts}.csv"))